In [ ]:
# 取得中文字型for juypter Notebook
import wget
f_url = 'https://github.com/flyingpath/electron-hand-dicom/raw/master/TaipeiSansTCBeta-Regular.ttf'
wget.download(f_url)

# 共通事前處理
# 隱藏不必要的警告
import warnings
warnings.filterwarnings('ignore')

# 匯入必要的函式庫
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm

# 將字型新增到 matplotlib
fm.fontManager.addfont('./TaipeiSansTCBeta-Regular.ttf')

# 用來顯示資料框的函式
from IPython.display import display

# 調整顯示選項
# NumPy 的浮點數表示精度
np.set_printoptions(suppress=True, precision=4)
# pandas 中的浮點數表示精度
pd.options.display.float_format = '{:.4f}'.format
# 顯示資料框中的所有項目
pd.set_option("display.max_columns",None)
# 指定圖形的預設字體大小
plt.rcParams["font.size"] = 14
# 指定圖形的預設字型
plt.rcParams['font.family'] = 'Taipei Sans TC Beta'
# 隨機種子
random_seed = 123

In [ ]:
#載入資料、確認資料
df = pd.read_csv('Bike-Sharing-Dataset/day.csv', parse_dates=[1]) #利用parse_dates指定日期所在的行，並將其轉為datetime

display(df.info()) #確認資料型態

df = df.drop('instant', axis=1) #先把無用處的編號欄位刪除
columns = [
    '日期',  '季節',  '年份', '月份', '國定假日', '星期幾', '工作日', '天氣', 
    '氣溫', '體感溫度',  '濕度', '風速',
    '臨時用戶租借量', '註冊用戶租借量', '整體用戶租借量'
]
df.columns = columns
display(df.head())
display(df.tail()) #確認結尾資料

In [ ]:
#繪製次數分佈圖，查看是否已標準化

#用來調整未來每一張圖形大小固定的方法，plt.figure()是指調整目前單一圖形大小
from pylab import rcParams
rcParams['figure.figsize'] = (12,12)

df.hist(bins=20) #直方圖的間格數(箱數)
plt.tight_layout()
plt.show()


In [ ]:
#處理遺失值
df.isnull().sum()

In [ ]:
#繪製時間序列資料圖，因為要預測時間的[註冊用戶租借量]
plt.figure(figsize=(12,4))
plt.plot(df['日期'], df['註冊用戶租借量'], color='b')
plt.title('註冊用戶租借量')

plt.grid() #顯示網格
plt.show()

In [ ]:
#先確認目標變數(租借量)、輸入變數、分割訓練資料與驗證資料
#日期分割成訓練與驗證，以20121101作為分界點，前22個月為訓練資料，後2個月為驗證資料
#分割輸入資料x與標準答案(目標變數y)

x = df.drop(['日期', '註冊用戶租借量', '臨時用戶租借量', '整體用戶租借量'], axis=1)
y = df['註冊用戶租借量'].values

In [ ]:
#設定分割日
mday = pd.to_datetime('2012-11-1') #轉換為Timestamp

#建立訓練用的index與驗證用的index
train_index = df['日期'] < mday #日期早於mday的資料索引
test_index = df['日期'] >= mday #日期晚於mday的資料索引

#分割輸入資料
x_train, x_test = x[train_index], x[test_index]

#分割目標資料
y_train, y_test = y[train_index], y[test_index]

#分割日期資料(之後用於繪製圖形)
dates_test = df['日期'][test_index]

#確認分割後的結果(大小、邊界)
print(x_train.shape)
print(x_test.shape)
display(x_train.tail()) #訓練資料的邊界(尾)
display(x_test.head()) #驗證資料的邊界(頭)

In [ ]:
#選擇演算法
#因為演算法很多，直接用XGBoost分類演算法的回歸版本，在回歸演算法中以效果優良著稱
from xgboost import XGBRegressor

#objective:squarederror是使用MSE均方誤差解決回歸問題,回歸問題中通常會用MSE當作損失函數。分類問題通常交叉熵(Cross entropy)當作損失函數
algorithm = XGBRegressor(objective ='reg:squarederror', random_state=random_seed)

In [ ]:
#訓練與預測
#訓練
algorithm.fit(x_train, y_train)
#預測
y_pred = algorithm.predict(x_test)

print(y_pred[:5]) #會輸出前5筆的預測租借數量


In [ ]:
#評估(使用R2決定係數評估)

#呼叫score函式
score = algorithm.score(x_test, y_test)

#計算R2值
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)

print(f'score: {score:.4f}  r2_score: {r2_score:.4f}')


In [ ]:
#評估(繪製散佈圖來評估)
plt.figure(figsize=(6,6))

y_max = y_test.max()
plt.plot((0, y_max), (0, y_max), color='k')
plt.scatter(y_test, y_pred, c='b')
plt.title(f'標準答案散佈圖(註冊用戶租借量) \ R2={r2_score:.4f}')
plt.grid()
plt.show()

In [ ]:
#繪製時間序列圖(註冊用戶租借量)
import matplotlib.dates as mdates
#建立畫布和子座標
fig, ax = plt.subplots(figsize=(8, 4))

#繪製圖形
ax.plot(dates_test, y_test, label='標準答案', c='k')
ax.plot(dates_test, y_pred, label='預測值', c='b')

#日期刻度間格，於每週四顯示日期
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

#將日期刻度標籤文字選轉90度
ax.tick_params(axis='x', rotation=90)

#顯示網格、圖利、標題
ax.grid()
ax.legend()
ax.set_title('預測註冊用戶租借量')

plt.show()


pd.concat(objs, axis=0, join='outer', ignore_index=False, keys=None)
objs 必需。要合併的 Pandas 物件序列。  list 或 dict (包含 Series 或 DataFrame)
axis 合併方向。決定是堆疊還是並排。  0 (預設值，按列堆疊，增加行數，UNION ALL)  1 (按行並排，增加列數，JOIN)
join 處理非重疊欄位/索引的方式。  'outer' (預設值，保留所有欄位/索引，缺失處填 NaN)  'inner' (只保留所有物件共有的欄位/索引)
ignore_index 是否忽略原始索引。  False (預設值，保留原始物件的索引)  True (忽略原始索引，重新產生 0 到 N-1 的新索引)
keys 為每個輸入物件加上層級索引。  list (提供名稱列表)

In [ ]:
#調整，因為月份、季節數值大小影響，讓模型差距較大，所以要one-hot編碼
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df.drop([column],axis=1),df_dummy], axis=1)
    return df

x2 = x.copy()
x2 = enc(x2, '月份')
x2 = enc(x2, '季節')

display(x2.head())